# 5-1 スクレイピング

* インストール

```
(pydataenv) $ pip install requests==2.32.3
(pydataenv) $ pip install beautifulsoup4==4.12.3
```

* Requests
  * 公式サイト: https://requests.readthedocs.io/
* BeautifulSoup4
  * 公式サイト: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
import requests

r = requests.get("https://www.seshop.com/")  # URLにアクセス
print(type(r))
print(r.status_code) # ステータスコードを確認

<class 'requests.models.Response'>
200


In [2]:
text = r.text  # HTMLのソースコードを取得する
for line in text.split("\n"):
    if "<title>" in line or "<h2>" in line:
        print(line.strip())

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>電子書籍ランキング<span class="pull-right"><a href="/ranking/327/"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>予約ランキング</h2>
<h2>新刊案内</h2>
<h2>SEshop限定　紙書籍と電子書籍のセット商品</h2>
<h2>商品カテゴリー</h2>
<h2>人気商品ランキング</h2>
<h2>キャンペーン・特集</h2>


In [3]:
from bs4 import BeautifulSoup

# HTMLを解析したBeautifulSoupオブジェクトを生成
soup = BeautifulSoup(text, "html.parser")
print(soup.title)  # <title> タグの情報を取得
print(soup.h2)  # <h2> タグの情報を取得
# h2タグの中のaタグのhref属性
print(soup.h2.a["href"])

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
/ranking/1


In [4]:
atags = soup.find_all("a")  # すべてのaタグを取得
print(f"aタグの数: {len(atags)}")  # aタグの数を取得
for atag in atags[:5]:
    print(f"タイトル: {atag.text}")  # aタグのテキストを取得
    print(f"リンク: {atag["href"]}")  # aタグのリンクを取得

aタグの数: 234
タイトル:  ヘルプ
リンク: /help
タイトル:  法人のお客様へ
リンク: /campaign/corp
タイトル:  新規会員登録
リンク: https://www.seshop.com/regist/
タイトル:  ログイン
リンク: #modalLogin
タイトル: 
リンク: /


In [5]:
from datetime import datetime

import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.seshop.com/product/616"
r = requests.get(BASE_URL)
soup = BeautifulSoup(r.text, "html.parser")

books = []  # 各書籍の情報を格納するリスト

# CSSセレクターで <div class="list">の中の
# <div class="inner"> を取得
divs = soup.select("div.list div.inner")
for div in divs:
    # 画像のURLを取得
    img_url = div.find("img")["src"]
    # 日付の文字列を取得
    day = div.find("span", class_="date").text.strip()
    day = day.removesuffix("発売")
    # 日付をdatetimeに変換
    published = datetime.strptime(day, "%Y.%m.%d")

    div_txt = div.find("div", class_="txt")
    a_tag = div_txt.find("a")  # aタグを取得
    title = a_tag.text.strip()  # 書籍タイトルを取得
    url = a_tag["href"]  # 書籍URLを取得

    # 販売価格を取得
    price_s = div_txt.find("span", class_="sale-price").parent.text
    price_s = price_s.split("円")[0]
    price_s = price_s.replace(",", "")
    price = int(price_s)

    book = {
        "title": title,
        "img_url": img_url,
        "url": url,
        "price": price,
        "published": published,
    }
    books.append(book)

In [6]:
books[:3]

[{'title': 'Pythonによるあたらしいデータ分析の教科書 第3版',
  'img_url': '/static/images/product/26879/L.png',
  'url': '/product/detail/26879',
  'price': 3058,
  'published': datetime.datetime(2025, 5, 21, 0, 0)},
 {'title': '独習Python 第2版',
  'img_url': '/static/images/product/26866/L.png',
  'url': '/product/detail/26866',
  'price': 3608,
  'published': datetime.datetime(2025, 5, 14, 0, 0)},
 {'title': '現場で使える！NumPyデータ処理入門 第2版  機械学習・データサイエンスで役立つ高速処理手法',
  'img_url': '/static/images/product/26383/L.png',
  'url': '/product/detail/26383',
  'price': 4400,
  'published': datetime.datetime(2024, 8, 26, 0, 0)}]

In [7]:
import pandas as pd

df = pd.DataFrame(books)  # 辞書をDataFrameに変換
df.head()

,title,img_url,url,price,published
0,Pythonによるあたらしいデータ分析の教科書 第3版,/static/images/product/26879/L.png,/product/detail/26879,3058,2025-05-21
1,独習Python 第2版,/static/images/product/26866/L.png,/product/detail/26866,3608,2025-05-14
2,現場で使える！NumPyデータ処理入門 第2版 機械学習・データサイエンスで役立つ高速処理手法,/static/images/product/26383/L.png,/product/detail/26383,4400,2024-08-26
3,動かして学ぶ！Python Django開発入門 第3版,/static/images/product/26360/L.png,/product/detail/26360,3960,2024-08-09
4,いきなりプログラミング Python,/static/images/product/26239/L.png,/product/detail/26239,2420,2024-06-25


In [8]:
import time
from urllib import parse

# HTML上の文字列とbook辞書に保存するときのキー名の対応表
LABELS = {
    "ISBN": "isbn",
    "判型": "format",
    "ページ数": "page",
}

for book in books:
    # 書籍の詳細ページのURLを作成してデータを取得
    url = parse.urljoin(BASE_URL, book["url"])
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    # 書籍の詳細情報を取得
    div = soup.find("div", class_="col-md-5")

    # 著者名を取得
    authors = []
    for a_tag in div.p.find_all("a"):
        authors.append(a_tag.text)
    book["author"] = authors

    # <dt>タグと<dd>タグから情報を取得
    dl = soup.find("dl")
    dt_list = dl.find_all("dt")
    dd_list = dl.find_all("dd")
    for dt, dd in zip(dt_list, dd_list):
        dt_text = dt.text.strip()
        # <dt>タグの名前が LABELS にあれば、
        # <dd>タグの内容をbook辞書に追加する
        if dt_text in LABELS:
            dd_text = dd.text.strip()
            # ページ数はint型に変換
            if dt_text == "ページ数":
                book[LABELS[dt_text]] = int(dd_text)
            else:
                book[LABELS[dt_text]] = dd_text

    # 1秒停止する
    time.sleep(1)

In [9]:
books[0]

{'title': 'Pythonによるあたらしいデータ分析の教科書 第3版',
 'img_url': '/static/images/product/26879/L.png',
 'url': '/product/detail/26879',
 'price': 3058,
 'published': datetime.datetime(2025, 5, 21, 0, 0),
 'author': ['寺田 学', '辻 真吾', '鈴木 たかのり', '福島 真太朗'],
 'isbn': '9784798191027',
 'format': 'A5',
 'page': 344}